In [2]:
import json

with open('credentials.json') as f:
    data = json.load(f)
    api_key = data['api_key']
    openai_api_key = data['openai_api_key']

In [3]:
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key=api_key)


/home/mchone/Documents/projects/pinecone-notebook/venv/lib/python3.11/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [4]:

with open("emoji.txt") as f:
    emoji = [line.split("#")[1].strip().split(" ", 2) for line in f if "fully-qualified" in line and not line.startswith("#")]

print(len(emoji))

3773


In [5]:
e = emoji[0]
print(e)

print(float(e[1].removeprefix("E")))


['😀', 'E1.0', 'grinning face']
1.0


In [6]:
index = pc.Index("emoji")


In [7]:
from openai import OpenAI

client = OpenAI(
    # This is the default and can be omitted
    api_key=openai_api_key,
)

In [8]:
def chunks(lst, n):
    """Yield successive chunks of size n """
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

descriptions = [f[2] for f in emoji if len(f) == 3]
embeddings = []

for chunk in list(chunks(descriptions, 1000)):
    print(len(chunk))
    e = client.embeddings.create(input=chunk, model="text-embedding-3-small")
    embeddings.extend(e.data)
    
print(len(embeddings))
print(len(emoji))

1000
1000
1000
772
3772
3773


In [12]:
vectors = []
i = 0
for e in emoji:
    if len(e) != 3:
        continue
    vectors.append({"id": e[0], "values": embeddings[i].embedding, "metadata": { "edition": float(e[1].removeprefix("E")), "name": e[2] }},)
    i = i + 1
    
print(vectors[0])
print(len(vectors))

{'id': '😀', 'values': [0.004887338, -0.024229012, 0.025765823, 0.027164184, -0.011477629, -0.015631175, 0.030874684, 0.026250403, -0.034889776, -0.021736884, 0.04535671, 0.020172384, 0.02366136, -0.035720486, 0.00022217138, 0.038738728, -0.049759466, -0.012785995, -0.0013230771, 0.01295906, 0.031760775, 0.011712996, 0.0037901097, 0.03148387, 0.017957158, 0.046547394, 0.013935143, -0.044941355, 0.027967202, -0.0036101227, -0.07232706, -0.04895645, 0.006635288, -0.07515147, 0.02868715, 0.029213266, 0.003540897, -0.0025907736, -0.015866542, -0.020490821, -0.005610747, -0.06806275, 0.015354271, 0.02078157, 0.027704144, -0.022941412, 0.008639373, 0.00863245, 0.044913664, 0.058537293, -0.016434193, 0.0006377422, -0.0027274946, 0.052390046, -0.05200238, 0.011789145, 0.013305189, 0.0058011175, -0.026610378, 0.016503418, 0.021833802, -0.020476976, 0.027468776, 0.033892926, -0.010660765, -0.03245303, -0.023439838, 0.023758277, -0.054134537, 0.042781513, 0.0018171759, -0.0029005588, 0.008750134, 

In [13]:
for v in vectors:
    v["metadata"]["emoji"] = v["id"]
    v["id"] = v["metadata"]["name"]
    v["id"] = v["id"].replace("’", "-").replace("ñ", "n").replace("“", "\"").replace("”", "\"").replace("Å", "A").replace("é", "e").replace("ô", "o").replace("ç", "c").replace("ã", "a").replace("í", "i").replace("ü", "u")
print(vectors[0])

{'id': 'grinning face', 'values': [0.004887338, -0.024229012, 0.025765823, 0.027164184, -0.011477629, -0.015631175, 0.030874684, 0.026250403, -0.034889776, -0.021736884, 0.04535671, 0.020172384, 0.02366136, -0.035720486, 0.00022217138, 0.038738728, -0.049759466, -0.012785995, -0.0013230771, 0.01295906, 0.031760775, 0.011712996, 0.0037901097, 0.03148387, 0.017957158, 0.046547394, 0.013935143, -0.044941355, 0.027967202, -0.0036101227, -0.07232706, -0.04895645, 0.006635288, -0.07515147, 0.02868715, 0.029213266, 0.003540897, -0.0025907736, -0.015866542, -0.020490821, -0.005610747, -0.06806275, 0.015354271, 0.02078157, 0.027704144, -0.022941412, 0.008639373, 0.00863245, 0.044913664, 0.058537293, -0.016434193, 0.0006377422, -0.0027274946, 0.052390046, -0.05200238, 0.011789145, 0.013305189, 0.0058011175, -0.026610378, 0.016503418, 0.021833802, -0.020476976, 0.027468776, 0.033892926, -0.010660765, -0.03245303, -0.023439838, 0.023758277, -0.054134537, 0.042781513, 0.0018171759, -0.0029005588, 0

In [15]:
index.upsert(vectors=vectors, batch_size=100)






























Upserted vectors:   0%|          | 0/3772 [01:29<?, ?it/s]










Upserted vectors: 100%|██████████| 3772/3772 [00:40<00:00, 93.87it/s]


{'upserted_count': 3772}

In [ ]:
with open("my.json","w") as f:
    json.dump(vectors,f)